### 기존 폴리건 영역을 만든 상태일 때 폴리건의 근접한 영역 100m를 계산하기 위한 코드
- <참고 URL> : https://drawing-polygon.netlify.app/


In [ ]:
import pandas as pd
from shapely import wkt
from shapely.geometry import Polygon, Point
from pyproj import CRS, Transformer

# --- 좌표계 변환 설정 ---
crs_wgs84 = CRS("EPSG:4326")
crs_web_mercator = CRS("EPSG:3857")
transformer_to_mercator = Transformer.from_crs(crs_wgs84, crs_web_mercator, always_xy=True)
transformer_from_mercator = Transformer.from_crs(crs_web_mercator, crs_wgs84, always_xy=True)

def transform_geometry(geometry, transformer):
    if geometry.is_empty:
        return geometry

    if isinstance(geometry, Polygon):
        exterior_coords = [transformer.transform(x, y) for x, y in geometry.exterior.coords]
        interior_coords_list = [
            [transformer.transform(x, y) for x, y in ring.coords]
            for ring in geometry.interiors
        ]
        return Polygon(exterior_coords, interior_coords_list)
    elif isinstance(geometry, Point):
        x, y = transformer.transform(geometry.x, geometry.y)
        return Point(x, y)
    else:
        return geometry.transform(lambda x, y: transformer.transform(x, y))

def calculate_buffered_and_donut_polygons(row, buffer_distance_m=100):
    original_wkt = row['polygon_str']
    
    try:
        original_poly_wgs84 = wkt.loads(original_wkt)
        original_poly_mercator = transform_geometry(original_poly_wgs84, transformer_to_mercator)
        
        buffered_poly_mercator = original_poly_mercator.buffer(buffer_distance_m)
        donut_poly_mercator = buffered_poly_mercator.difference(original_poly_mercator)

        buffered_wkt = transform_geometry(buffered_poly_mercator, transformer_from_mercator).wkt
        donut_wkt = transform_geometry(donut_poly_mercator, transformer_from_mercator).wkt
        
        return buffered_wkt, donut_wkt

    except Exception as e:
        print(f"Error processing {row['campus_name']}: {e}")
        return None, None

# --- 파일 경로 설정 ---
input_file_path = 'campus_polygons.csv'
output_file_path = 'campus_polygons_updated.csv'

try:
    df = pd.read_csv(input_file_path)
    print(f"'{input_file_path}' 파일을 성공적으로 읽었습니다.")
    print("원본 데이터:")
    print(df.head())

    df[['WKT_Polygon_100m_Buffered', 'WKT_Polygon_100m_Donut']] = df.apply(
        lambda row: calculate_buffered_and_donut_polygons(row), axis=1, result_type='expand'
    )

    df = df.drop(columns=['WKT_Polygon_100m_Buffered'])  # 이 컬럼이 필요 없다면 제거
    print("\n계산 완료. 업데이트된 데이터:")
    print(df.head())

    # --- 쉼표로 구분된 CSV로 저장 ---
    df.to_csv(output_file_path, index=False)
    print(f"\n업데이트된 데이터가 '{output_file_path}'에 성공적으로 저장되었습니다.")

except FileNotFoundError:
    print(f"오류: '{input_file_path}' 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
except Exception as e:
    print(f"데이터 처리 중 오류 발생: {e}")

'seoul_hotspots_polygons.csv' 파일을 성공적으로 읽었습니다.
원본 데이터:
  location                                        polygon_str
0     강남역1  POLYGON((127.021136 37.503349, 127.026467 37.5...
1     강남역2  POLYGON((127.017214 37.507406, 127.019084 37.5...
2       건대  POLYGON ((127.066675 37.5469, 127.069465 37.54...
3      광화문  POLYGON ((126.971937 37.570065, 126.968866 37....
4      대학로  POLYGON((126.997377 37.575873, 126.996574 37.5...

계산 완료. 업데이트된 데이터:
  location                                        polygon_str  \
0     강남역1  POLYGON((127.021136 37.503349, 127.026467 37.5...   
1     강남역2  POLYGON((127.017214 37.507406, 127.019084 37.5...   
2       건대  POLYGON ((127.066675 37.5469, 127.069465 37.54...   
3      광화문  POLYGON ((126.971937 37.570065, 126.968866 37....   
4      대학로  POLYGON((126.997377 37.575873, 126.996574 37.5...   

                              WKT_Polygon_100m_Donut  
0  POLYGON ((127.02026854757358 37.50316380358851...  
1  POLYGON ((127.01633049310564 37.50727714239723... 